In [7]:
import scanpy as sc 
import pandas as pd 
import helper_functions

In [2]:
adata= sc.read_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Linnarson/Linnarson_unfiltered.h5ad')

In [3]:
adata

AnnData object with n_obs × n_vars = 3892388 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier'

In [8]:
metadata= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/sample_metadata/Linnarson_Metadata_Final.csv',index_col=0)

In [9]:
metadata

,accession_id,alias,Sample_ID,Age,Donor_Anatomical_Site_Age,Sex,Donor_ID,Disease_Status,extra_attributes,Organ,...,Accession_ID,Method,Anatomical_Site,10X_Chemistry,Age(PCW),Organism,Multiplexed,Expected_Donors,FACS_Sorting,Dataset
0,EGAN00002408138,10X147_5_AB_1,10X147_5,6.0,XDD:326_Spinal_cord_lumbar_6w,unknown,XDD:326,normal,EGA00001631142,Brain,...,EGAD00001006049,3GEX,Spinal_cord_lumbar,10x3'v2,6.0,Human,No,1,Total,Linnarson
1,EGAN00002408139,10X101_8_GRCh38-3_0_0,10X101_8,10.0,XHU:297_Forebrain_10w,unknown,XHU:297,normal,EGA00001631144,Brain,...,EGAD00001006049,3GEX,Forebrain,10x3'v2,10.0,Human,No,1,Total,Linnarson
2,EGAN00002408140,10X101_7_GRCh38-3_0_0,10X101_7,10.0,XHU:297_Forebrain_10w,unknown,XHU:297,normal,EGA00001631145,Brain,...,EGAD00001006049,3GEX,Forebrain,10x3'v2,10.0,Human,No,1,Total,Linnarson
3,EGAN00002408141,10X101_6_GRCh38-3_0_0,10X101_6,10.0,XHU:297_Forebrain_10w,unknown,XHU:297,normal,EGA00001631146,Brain,...,EGAD00001006049,3GEX,Forebrain,10x3'v2,10.0,Human,No,1,Total,Linnarson
4,EGAN00002408142,10X101_5_GRCh38-3_0_0,10X101_5,10.0,XHU:297_Forebrain_10w,unknown,XHU:297,normal,EGA00001631147,Brain,...,EGAD00001006049,3GEX,Forebrain,10x3'v2,10.0,Human,No,1,Total,Linnarson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,EGAN50000223127,10X254_8_AB_1,10X254_8,14.0,XDD:385_Thalamus_14w,unknown,XDD:385,normal,[{}],Brain,...,EGAD50000001295,3GEX,Thalamus,10x3'v3,14.0,Human,No,1,Total,Linnarson
444,EGAN50000223128,10X254_4_AB_1,10X254_4,14.0,XDD:385_Cerebellum_14w,unknown,XDD:385,normal,[{}],Brain,...,EGAD50000001295,3GEX,Cerebellum,10x3'v3,14.0,Human,No,1,Total,Linnarson
445,EGAN50000223129,10X302_2_AB_1,10X302_2,7.0,XDD:398_Hindbrain_7w,unknown,XDD:398,normal,[{}],Brain,...,EGAD50000001295,3GEX,Hindbrain,10x3'v3,7.0,Human,No,1,Total,Linnarson
446,EGAN50000223130,10X298_4_AB_1,10X298_4,5.5,XDD:400_Hindbrain_5.5w,unknown,XDD:400,normal,[{}],Brain,...,EGAD50000001295,3GEX,Hindbrain,10x3'v3,5.0,Human,No,1,Total,Linnarson


In [5]:
import pandas as pd

def add_metadata_to_obs(
    adata,
    df: pd.DataFrame,
    df_key: str,
    obs_key: str = None,
):
    """
    Merge EVERY column of `df` into `adata.obs` by matching
      df[df_key]  ↔  adata.obs[obs_key] (or adata.obs_names if obs_key is None).

    Keeps duplicates in df_key by dropping all but the first.
    Leaves adata.obs index untouched.
    """

    # 1. Sanity checks
    assert df_key in df.columns, f"{df_key!r} not in metadata DataFrame"
    if obs_key is not None:
        assert obs_key in adata.obs.columns, f"{obs_key!r} not in adata.obs"

    # 2. Deduplicate metadata on the key, keep only first
    df_unique = df.drop_duplicates(subset=[df_key], keep="first").copy()

    # 3. Build a lookup Series indexed by the key
    df_indexed = df_unique.set_index(df_key)

    # 4. Pick the “left” key from adata.obs
    if obs_key is None:
        # map from obs_names
        left_index = pd.Series(adata.obs_names, index=adata.obs_names, name=df_key)
    else:
        # map from an existing obs column
        left_index = adata.obs[obs_key].rename(df_key)

    # 5. For every column in df_indexed (including df_key itself), map into adata.obs
    for col in df_indexed.columns:
        adata.obs[col] = left_index.map(df_indexed[col])

    # 6. Report if anything expected went missing
    missing = adata.obs[df_indexed.columns].isna().any(axis=1).sum()
    if missing:
        print(f"⚠️  {missing} cells did not get full metadata from `{df_key}` join")

    return adata


In [6]:
 #)add metadata keyed by Sample_ID → obs['Sanger_ID']
add_metadata_to_obs(adata, metadata,   df_key='Sample_ID', obs_key='Sanger_ID')


⚠️  461016 cells did not get full metadata from `Sample_ID` join


AnnData object with n_obs × n_vars = 3892388 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'accession_id', 'alias', 'Age', 'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status', 'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID', 'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism', 'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset'

In [7]:
samples=list(adata.obs["Sanger_ID"].unique())

# Filter out low cell count, low gene count and high mitochondria count cells 

In [1]:
import scanpy as sc 
import pandas as pd 
import helper_functions

In [7]:
adata[adata.obs['mt_outlier']==True]

View of AnnData object with n_obs × n_vars = 60955 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'accession_id', 'alias', 'Age', 'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status', 'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID', 'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism', 'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset'

In [8]:
import numpy as np
from scipy.stats import median_abs_deviation

# 1) make sure you're using the correct metric – usually Scanpy gives you 'pct_counts_mt'
metric = "log1p_total_counts_mt"

# 2) compute the per‐sample threshold
def upper_thresh(x, nmads=3):
    return x.median() + nmads * median_abs_deviation(x)

# this will give you, for each row, the threshold of its sample
th = (
    adata
    .obs
    .groupby("Sanger_ID")[metric]
    .transform(lambda x: upper_thresh(x, nmads=3))
)

# 3) flag anything above that threshold
adata.obs["mt_high_outlier"] = adata.obs[metric] > th


In [9]:
adata[adata.obs['mt_high_outlier']==True]

View of AnnData object with n_obs × n_vars = 60955 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'accession_id', 'alias', 'Age', 'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status', 'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID', 'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism', 'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset', 'mt_high_outlier'

In [10]:
conditions = [
    (adata.obs['total_counts_outlier']== True),
    (adata.obs['gene_counts_outlier'] == True),
     (adata.obs['mt_outlier'] == True),
    (
    (adata.obs['gene_counts_outlier']== False) & (adata.obs['gene_counts_outlier'] == False) & (adata.obs['mt_high_outlier']==False))
   
]
values = ['Low_total_count', 'Low_nFeature', "High_Mito",'Pass']

In [11]:
adata.obs['QC'] = np.select(conditions, values, default="Pass")

In [12]:
adata.obs['QC'].value_counts()

QC
Pass               3350183
Low_total_count     313361
Low_nFeature        174422
High_Mito            54422
Name: count, dtype: int64

In [13]:
adata.obs["mt_high_outlier"].value_counts()

mt_high_outlier
False    3831433
True       60955
Name: count, dtype: int64

In [14]:
pd.crosstab(adata.obs['mt_high_outlier'],adata.obs['QC'])

QC,High_Mito,Low_nFeature,Low_total_count,Pass
mt_high_outlier,,,,
False,0,174208,307042,3350183
True,54422,214,6319,0


In [15]:
pct = (
    pd.crosstab(
        adata.obs['Technology'].copy(),
        adata.obs['mt_high_outlier'].copy(),
        normalize='index'
    )
    * 100
)

# rename the columns for clarity
pct.columns = ['Non‐Outlier (%)', 'Mito‐Outlier (%)']


In [16]:
adata

AnnData object with n_obs × n_vars = 3892388 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'accession_id', 'alias', 'Age', 'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status', 'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID', 'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism', 'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset', 'mt_high_outlier', 'QC'

In [18]:
adata[adata.obs['total_counts']>500]

View of AnnData object with n_obs × n_vars = 2843945 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'accession_id', 'alias', 'Age', 'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status', 'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID', 'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism', 'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset', 'mt_high_outlier', 'QC'

View of AnnData object with n_obs × n_vars = 2368170 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'accession_id', 'alias', 'Age', 'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status', 'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID', 'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism', 'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset', 'mt_high_outlier', 'QC'

In [14]:
adata=adata[adata.obs['QC']=="Pass"].copy()

In [27]:
adata[adata.obs["Sanger_ID"]=='10X109_1']

View of AnnData object with n_obs × n_vars = 9921 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'accession_id', 'alias', 'Age', 'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status', 'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID', 'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism', 'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset', 'mt_high_outlier', 'QC'

In [28]:
# Remove failed samples

In [29]:
linnarson_paper_metadata= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/sample_metadata/Linnarson_Metadata_from_Paper.csv')

In [31]:
linnarson_paper_metadata["QC"]

0      Passed
1      Passed
2      Passed
3      Passed
4      Passed
        ...  
403    Failed
404    Failed
405    Failed
406    Failed
407    Failed
Name: QC, Length: 408, dtype: object

In [35]:
# get the list of Sample_IDs that failed
failed_ids = linnarson_paper_metadata.loc[linnarson_paper_metadata["QC"] == "Failed", "Sample_ID"].unique().tolist()

print(f"Found {len(failed_ids)} failed samples:", failed_ids)

# if you want to subset your AnnData to just the failed samples:
adata_failed = adata[adata.obs["Sanger_ID"].isin(failed_ids)].copy()
print(f"adata_failed has {adata_failed.n_obs} cells across {len(failed_ids)} samples")


Found 66 failed samples: ['10X109_5', '10X109_1', '10X154_5', '10X154_6', '10X154_7', '10X169_3', '10X122_4', '10X196_3', '10X196_4', '10X197_3', '10X197_4', '10X96_5', '10X96_6', '10X96_1', '10X96_2', '10X97_1', '10X97_2', '10X98_1', '10X98_2', '10X98_3', '10X98_4', '10X99_1', '10X99_2', '10X96_7', '10X96_8', '10X97_3', '10X97_4', '10X96_3', '10X96_4', '10X99_3', '10X99_4', '10X98_5', '10X98_6', '10X155_1', '10X155_3', '10X105_1', '10X105_2', '10X105_3', '10X103_1', '10X103_2', '10X103_3', '10X104_2', '10X143_1', '10X143_2', '10X163_5', '10X163_6', '10X165_2', '10X115_4', '10X114_2', '10X114_5', '10X114_6', '10X114_7', '10X114_8', '10X185_3', '10X186_2', '10X231_1', '10X231_2', '10X231_5', '10X231_6', '10X257_3', '10X255_5', '10X258_3', '10X258_4', '10X258_1', '10X258_2', '10X262_5']
adata_failed has 752087 cells across 66 samples


In [41]:
failed_ids

['10X109_5',
 '10X109_1',
 '10X154_5',
 '10X154_6',
 '10X154_7',
 '10X169_3',
 '10X122_4',
 '10X196_3',
 '10X196_4',
 '10X197_3',
 '10X197_4',
 '10X96_5',
 '10X96_6',
 '10X96_1',
 '10X96_2',
 '10X97_1',
 '10X97_2',
 '10X98_1',
 '10X98_2',
 '10X98_3',
 '10X98_4',
 '10X99_1',
 '10X99_2',
 '10X96_7',
 '10X96_8',
 '10X97_3',
 '10X97_4',
 '10X96_3',
 '10X96_4',
 '10X99_3',
 '10X99_4',
 '10X98_5',
 '10X98_6',
 '10X155_1',
 '10X155_3',
 '10X105_1',
 '10X105_2',
 '10X105_3',
 '10X103_1',
 '10X103_2',
 '10X103_3',
 '10X104_2',
 '10X143_1',
 '10X143_2',
 '10X163_5',
 '10X163_6',
 '10X165_2',
 '10X115_4',
 '10X114_2',
 '10X114_5',
 '10X114_6',
 '10X114_7',
 '10X114_8',
 '10X185_3',
 '10X186_2',
 '10X231_1',
 '10X231_2',
 '10X231_5',
 '10X231_6',
 '10X257_3',
 '10X255_5',
 '10X258_3',
 '10X258_4',
 '10X258_1',
 '10X258_2',
 '10X262_5']

In [39]:
adata=adata[(adata.obs['QC']=="Pass") & (adata.obs['n_genes_by_counts']>200) & (adata.obs['total_counts']<60000)& (adata.obs['total_counts']>1000) ]

In [40]:
adata

View of AnnData object with n_obs × n_vars = 2368170 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'accession_id', 'alias', 'Age', 'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status', 'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID', 'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism', 'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset', 'mt_high_outlier', 'QC'

In [ ]:
# failed_ids = list of Sample_IDs with QC == "Fail"
# build mask of all cells NOT in a failed sample
keep_mask = ~adata.obs["Sample_ID"].isin(failed_ids)

# subset
adata_pass = adata[keep_mask].copy()


In [1]:
import scanpy as sc

In [5]:
adata= sc.read_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Linnarson/Linnarson_filtered_with_metadata.h5ad')

In [11]:
adata

AnnData object with n_obs × n_vars = 2643336 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Age', 'Age(PCW)', 'Dissected_region', 'Target cell number', 'Retrieved cell number', 'Median genes/cell', 'Median UMIs/cell', 'QC', 'Chemistry', 'Source', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'mt_high_outlier'

In [10]:
metadata.columns

Index(['accession_id', 'alias', 'Sample_ID', 'Age',
       'Donor_Anatomical_Site_Age', 'Sex', 'Donor_ID', 'Disease_Status',
       'extra_attributes', 'Organ', 'Technology', 'Study', 'Accession_ID',
       'Method', 'Anatomical_Site', '10X_Chemistry', 'Age(PCW)', 'Organism',
       'Multiplexed', 'Expected_Donors', 'FACS_Sorting', 'Dataset'],
      dtype='object')